In [1]:
import pandas as pd
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, Dense, GlobalMaxPooling1D

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('data/train.csv')
val_data = pd.read_csv('data/validation.csv')
print(len(train_data), len(val_data))

105000 15000


In [3]:
train_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2


In [20]:
test_data = pd.read_csv('data/test.csv')
test_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_data.columns[2:]

Index(['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

- 位置：3个，location_traffic_convenience, location_distance_from_business_district, location_easy_to_find
- 服务：4个，service_wait_time, service_waiters_attitude, service_parking_convenience, service_serving_speed
- 价格：3个，price_level, price_cost_effective, price_discount
- 环境：4个，environment_decoration, environment_noise, environment_space, environment_cleaness
- 菜品：4个，dish_portion, dish_taste, dish_look, dish_recommendation
- 其他：2个，others_overall_experience, others_willing_to_consume_again

In [43]:
x_train = list(map(lambda x: x[1:-1], train_data['content']))
x_val = list(map(lambda x: x[1:-1], val_data['content']))

In [44]:
train_len = list(map(len, x_train))
np.max(train_len), np.min(train_len), np.mean(train_len), np.median(train_len), 

(2002, 142, 346.7424380952381, 283.0)

In [45]:
val_len = list(map(len, x_val))
np.max(val_len), np.min(val_len), np.mean(val_len), np.median(val_len)

(2000, 195, 345.13866666666667, 281.0)

In [46]:
maxlen = 1000
max_words = 8000

In [47]:
tokenizer = Tokenizer(num_words=max_words, char_level=True)
tokenizer.fit_on_texts(x_train)

In [48]:
sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=maxlen)

sequences = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(sequences, maxlen=maxlen)

In [49]:
x_test = list(map(lambda x: x[1:-1], test_data['content']))
sequences = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences, maxlen=maxlen)

In [50]:
y_train_ltc = pd.get_dummies(train_data['location_traffic_convenience'])[[-2, -1, 0, 1]].values
y_train_ldfbd = pd.get_dummies(train_data['location_distance_from_business_district'])[[-2, -1, 0, 1]].values
y_train_letf = pd.get_dummies(train_data['location_easy_to_find'])[[-2, -1, 0, 1]].values

y_train_swt = pd.get_dummies(train_data['service_wait_time'])[[-2, -1, 0, 1]].values
y_train_swa = pd.get_dummies(train_data['service_waiters_attitude'])[[-2, -1, 0, 1]].values
y_train_spc = pd.get_dummies(train_data['service_parking_convenience'])[[-2, -1, 0, 1]].values
y_train_sss = pd.get_dummies(train_data['service_serving_speed'])[[-2, -1, 0, 1]].values

y_train_pl = pd.get_dummies(train_data['price_level'])[[-2, -1, 0, 1]].values
y_train_pce = pd.get_dummies(train_data['price_cost_effective'])[[-2, -1, 0, 1]].values
y_train_pd = pd.get_dummies(train_data['price_discount'])[[-2, -1, 0, 1]].values

y_train_ed = pd.get_dummies(train_data['environment_decoration'])[[-2, -1, 0, 1]].values
y_train_en = pd.get_dummies(train_data['environment_noise'])[[-2, -1, 0, 1]].values
y_train_es = pd.get_dummies(train_data['environment_space'])[[-2, -1, 0, 1]].values
y_train_ec = pd.get_dummies(train_data['environment_cleaness'])[[-2, -1, 0, 1]].values

y_train_dp = pd.get_dummies(train_data['dish_portion'])[[-2, -1, 0, 1]].values
y_train_dt = pd.get_dummies(train_data['dish_taste'])[[-2, -1, 0, 1]].values
y_train_dl = pd.get_dummies(train_data['dish_look'])[[-2, -1, 0, 1]].values
y_train_dr = pd.get_dummies(train_data['dish_recommendation'])[[-2, -1, 0, 1]].values

y_train_ooe = pd.get_dummies(train_data['others_overall_experience'])[[-2, -1, 0, 1]].values
y_train_owtca = pd.get_dummies(train_data['others_willing_to_consume_again'])[[-2, -1, 0, 1]].values

In [51]:
y_val_ltc = pd.get_dummies(val_data['location_traffic_convenience'])[[-2, -1, 0, 1]].values
y_val_ldfbd = pd.get_dummies(val_data['location_distance_from_business_district'])[[-2, -1, 0, 1]].values
y_val_letf = pd.get_dummies(val_data['location_easy_to_find'])[[-2, -1, 0, 1]].values

y_val_swt = pd.get_dummies(val_data['service_wait_time'])[[-2, -1, 0, 1]].values
y_val_swa = pd.get_dummies(val_data['service_waiters_attitude'])[[-2, -1, 0, 1]].values
y_val_spc = pd.get_dummies(val_data['service_parking_convenience'])[[-2, -1, 0, 1]].values
y_val_sss = pd.get_dummies(val_data['service_serving_speed'])[[-2, -1, 0, 1]].values

y_val_pl = pd.get_dummies(val_data['price_level'])[[-2, -1, 0, 1]].values
y_val_pce = pd.get_dummies(val_data['price_cost_effective'])[[-2, -1, 0, 1]].values
y_val_pd = pd.get_dummies(val_data['price_discount'])[[-2, -1, 0, 1]].values

y_val_ed = pd.get_dummies(val_data['environment_decoration'])[[-2, -1, 0, 1]].values
y_val_en = pd.get_dummies(val_data['environment_noise'])[[-2, -1, 0, 1]].values
y_val_es = pd.get_dummies(val_data['environment_space'])[[-2, -1, 0, 1]].values
y_val_ec = pd.get_dummies(val_data['environment_cleaness'])[[-2, -1, 0, 1]].values

y_val_dp = pd.get_dummies(val_data['dish_portion'])[[-2, -1, 0, 1]].values
y_val_dt = pd.get_dummies(val_data['dish_taste'])[[-2, -1, 0, 1]].values
y_val_dl = pd.get_dummies(val_data['dish_look'])[[-2, -1, 0, 1]].values
y_val_dr = pd.get_dummies(val_data['dish_recommendation'])[[-2, -1, 0, 1]].values

y_val_ooe = pd.get_dummies(val_data['others_overall_experience'])[[-2, -1, 0, 1]].values
y_val_owtca = pd.get_dummies(val_data['others_willing_to_consume_again'])[[-2, -1, 0, 1]].values

In [52]:
def build_model():
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=maxlen))
    model.add(Conv1D(32, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
    return model

#### 位置

In [54]:
model1 = build_model()
model1.fit(x_train, y_train_ltc, epochs=3, batch_size=64, validation_data=(x_val, y_val_ltc))

Train on 105000 samples, validate on 15000 samples
Epoch 1/3
105000/105000 [==============================] - 25s 242us/step - loss: 0.2517 - acc: 0.9284 - val_loss: 0.2067 - val_acc: 0.9406
Epoch 2/3
105000/105000 [==============================] - 25s 238us/step - loss: 0.2065 - acc: 0.9399 - val_loss: 0.1936 - val_acc: 0.9438
Epoch 3/3
105000/105000 [==============================] - 25s 237us/step - loss: 0.1949 - acc: 0.9423 - val_loss: 0.2024 - val_acc: 0.9430


In [55]:
labels = [-2, -1, 0, 1]
ltc_preds = model1.predict_classes(x_test)
ltc_preds = [labels[i] for i in ltc_preds]

In [73]:
maxlen = 1000
max_words = 8000

x_train = list(map(lambda x: x[1:-1], train_data['content']))
x_val = list(map(lambda x: x[1:-1], val_data['content']))

tokenizer = Tokenizer(num_words=max_words, char_level=True)
tokenizer.fit_on_texts(x_train)

sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=maxlen)

sequences = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(sequences, maxlen=maxlen)

x_test = list(map(lambda x: x[1:-1], test_data['content']))
sequences = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(sequences, maxlen=maxlen)

In [74]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 64, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])

model1.fit(x_train, y_train_ldfbd, epochs=2, batch_size=64, validation_data=(x_val, y_val_ldfbd))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 255us/step - loss: 0.3597 - acc: 0.8742 - val_loss: 0.3249 - val_acc: 0.8807
Epoch 2/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.3127 - acc: 0.8877 - val_loss: 0.3120 - val_acc: 0.8830


In [75]:
labels = [-2, -1, 0, 1]
ldfbd_preds = model1.predict_classes(x_test)
ldfbd_preds = [labels[i] for i in ldfbd_preds]

In [77]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 64, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])

model1.fit(x_train, y_train_letf, epochs=2, batch_size=64, validation_data=(x_val, y_val_letf))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 258us/step - loss: 0.3847 - acc: 0.8858 - val_loss: 0.3215 - val_acc: 0.9053
Epoch 2/2
105000/105000 [==============================] - 26s 248us/step - loss: 0.3007 - acc: 0.9078 - val_loss: 0.2916 - val_acc: 0.9101


In [78]:
labels = [-2, -1, 0, 1]
letf_preds = model1.predict_classes(x_test)
letf_preds = [labels[i] for i in letf_preds]

#### 服务

In [79]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 64, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])

model1.fit(x_train, y_train_swt, epochs=2, batch_size=64, validation_data=(x_val, y_val_swt))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 260us/step - loss: 0.3350 - acc: 0.8927 - val_loss: 0.2963 - val_acc: 0.9020
Epoch 2/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.2905 - acc: 0.9061 - val_loss: 0.2887 - val_acc: 0.9050


In [80]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
swt_preds = [labels[i] for i in preds]

In [86]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])

model1.fit(x_train, y_train_swa, epochs=3, batch_size=64, validation_data=(x_val, y_val_swa))

Train on 105000 samples, validate on 15000 samples
Epoch 1/3
105000/105000 [==============================] - 35s 329us/step - loss: 0.5735 - acc: 0.7860 - val_loss: 0.4792 - val_acc: 0.8168
Epoch 2/3
105000/105000 [==============================] - 34s 322us/step - loss: 0.4498 - acc: 0.8330 - val_loss: 0.4492 - val_acc: 0.8294
Epoch 3/3
105000/105000 [==============================] - 34s 321us/step - loss: 0.4105 - acc: 0.8485 - val_loss: 0.4481 - val_acc: 0.8302


In [87]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
swa_preds = [labels[i] for i in preds]

In [88]:
model1 = build_model()
model1.fit(x_train, y_train_spc, epochs=2, batch_size=64, validation_data=(x_val, y_val_spc))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 254us/step - loss: 0.1148 - acc: 0.9643 - val_loss: 0.0883 - val_acc: 0.9725
Epoch 2/2
105000/105000 [==============================] - 25s 243us/step - loss: 0.0845 - acc: 0.9733 - val_loss: 0.0817 - val_acc: 0.9739


In [89]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
spc_preds = [labels[i] for i in preds]

In [90]:
model1 = build_model()
model1.fit(x_train, y_train_sss, epochs=3, batch_size=64, validation_data=(x_val, y_val_sss))

Train on 105000 samples, validate on 15000 samples
Epoch 1/3
105000/105000 [==============================] - 27s 253us/step - loss: 0.3457 - acc: 0.8998 - val_loss: 0.2820 - val_acc: 0.9144
Epoch 2/3
105000/105000 [==============================] - 26s 243us/step - loss: 0.2484 - acc: 0.9226 - val_loss: 0.2547 - val_acc: 0.9198
Epoch 3/3
105000/105000 [==============================] - 26s 244us/step - loss: 0.2305 - acc: 0.9276 - val_loss: 0.2497 - val_acc: 0.9245


In [91]:
model1.fit(x_train, y_train_sss, epochs=1, batch_size=64, validation_data=(x_val, y_val_sss))

Train on 105000 samples, validate on 15000 samples
Epoch 1/1
105000/105000 [==============================] - 26s 243us/step - loss: 0.2185 - acc: 0.9316 - val_loss: 0.2540 - val_acc: 0.9247


In [92]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
sss_preds = [labels[i] for i in preds]

#### 价格

In [99]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])

model1.fit(x_train, y_train_pl, epochs=2, batch_size=128, validation_data=(x_val, y_val_pl))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 39s 374us/step - loss: 0.6842 - acc: 0.7316 - val_loss: 0.5638 - val_acc: 0.7801
Epoch 2/2
105000/105000 [==============================] - 38s 362us/step - loss: 0.5267 - acc: 0.7989 - val_loss: 0.5417 - val_acc: 0.7925


In [100]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
pl_preds = [labels[i] for i in preds]

In [102]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_pce, epochs=2, batch_size=64, validation_data=(x_val, y_val_pce))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 45s 431us/step - loss: 0.3895 - acc: 0.8676 - val_loss: 0.3476 - val_acc: 0.8802
Epoch 2/2
105000/105000 [==============================] - 45s 433us/step - loss: 0.3364 - acc: 0.8835 - val_loss: 0.3324 - val_acc: 0.8844


In [103]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
pce_preds = [labels[i] for i in preds]

In [104]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_pd, epochs=2, batch_size=64, validation_data=(x_val, y_val_pd))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 45s 433us/step - loss: 0.5102 - acc: 0.8072 - val_loss: 0.4578 - val_acc: 0.8285
Epoch 2/2
105000/105000 [==============================] - 44s 419us/step - loss: 0.4419 - acc: 0.8341 - val_loss: 0.4479 - val_acc: 0.8357


In [105]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
pd_preds = [labels[i] for i in preds]

#### 环境

In [106]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_ed, epochs=2, batch_size=64, validation_data=(x_val, y_val_ed))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 435us/step - loss: 0.5316 - acc: 0.8150 - val_loss: 0.4500 - val_acc: 0.8440
Epoch 2/2
105000/105000 [==============================] - 44s 423us/step - loss: 0.4447 - acc: 0.8455 - val_loss: 0.4271 - val_acc: 0.8503


In [107]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
ed_preds = [labels[i] for i in preds]

In [108]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_en, epochs=2, batch_size=64, validation_data=(x_val, y_val_en))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 436us/step - loss: 0.4756 - acc: 0.8364 - val_loss: 0.4012 - val_acc: 0.8589
Epoch 2/2
105000/105000 [==============================] - 44s 423us/step - loss: 0.3967 - acc: 0.8630 - val_loss: 0.4109 - val_acc: 0.8612


In [109]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
en_preds = [labels[i] for i in preds]

In [110]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_es, epochs=2, batch_size=64, validation_data=(x_val, y_val_es))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 435us/step - loss: 0.6069 - acc: 0.7834 - val_loss: 0.4982 - val_acc: 0.8227
Epoch 2/2
105000/105000 [==============================] - 44s 421us/step - loss: 0.4864 - acc: 0.8270 - val_loss: 0.4908 - val_acc: 0.8264


In [111]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
es_preds = [labels[i] for i in preds]

In [112]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_ec, epochs=2, batch_size=64, validation_data=(x_val, y_val_ec))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 437us/step - loss: 0.5118 - acc: 0.8286 - val_loss: 0.4223 - val_acc: 0.8553
Epoch 2/2
105000/105000 [==============================] - 45s 427us/step - loss: 0.4201 - acc: 0.8583 - val_loss: 0.4063 - val_acc: 0.8649


In [113]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
ec_preds = [labels[i] for i in preds]

#### 菜品

In [114]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_dp, epochs=2, batch_size=64, validation_data=(x_val, y_val_dp))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 437us/step - loss: 0.7156 - acc: 0.7367 - val_loss: 0.6294 - val_acc: 0.7745
Epoch 2/2
105000/105000 [==============================] - 45s 424us/step - loss: 0.5937 - acc: 0.7832 - val_loss: 0.6100 - val_acc: 0.7777


In [115]:
model1.fit(x_train, y_train_dp, epochs=1, batch_size=64, validation_data=(x_val, y_val_dp))

Train on 105000 samples, validate on 15000 samples
Epoch 1/1
105000/105000 [==============================] - 44s 422us/step - loss: 0.5616 - acc: 0.7933 - val_loss: 0.6062 - val_acc: 0.7797


In [116]:
model1.fit(x_train, y_train_dp, epochs=1, batch_size=64, validation_data=(x_val, y_val_dp))

Train on 105000 samples, validate on 15000 samples
Epoch 1/1
105000/105000 [==============================] - 44s 421us/step - loss: 0.5357 - acc: 0.8020 - val_loss: 0.5996 - val_acc: 0.7801


In [117]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dp_preds = [labels[i] for i in preds]

In [118]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_dt, epochs=2, batch_size=64, validation_data=(x_val, y_val_dt))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 437us/step - loss: 0.6630 - acc: 0.7150 - val_loss: 0.5837 - val_acc: 0.7466
Epoch 2/2
105000/105000 [==============================] - 45s 424us/step - loss: 0.5484 - acc: 0.7683 - val_loss: 0.5449 - val_acc: 0.7652


In [119]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dt_preds = [labels[i] for i in preds]

In [120]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_dl, epochs=2, batch_size=64, validation_data=(x_val, y_val_dl))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 439us/step - loss: 0.6511 - acc: 0.7774 - val_loss: 0.5882 - val_acc: 0.7945
Epoch 2/2
105000/105000 [==============================] - 45s 424us/step - loss: 0.5713 - acc: 0.7993 - val_loss: 0.5765 - val_acc: 0.7981


In [121]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dl_preds = [labels[i] for i in preds]

In [122]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_dr, epochs=2, batch_size=64, validation_data=(x_val, y_val_dr))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 46s 442us/step - loss: 0.3731 - acc: 0.8829 - val_loss: 0.3327 - val_acc: 0.8941
Epoch 2/2
105000/105000 [==============================] - 44s 423us/step - loss: 0.3220 - acc: 0.8994 - val_loss: 0.3256 - val_acc: 0.9007


In [123]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
dr_preds = [labels[i] for i in preds]

#### 其他

In [135]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(32, 5, activation='relu'))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_ooe, epochs=4, batch_size=64, validation_data=(x_val, y_val_ooe))

Train on 105000 samples, validate on 15000 samples
Epoch 1/4
105000/105000 [==============================] - 37s 352us/step - loss: 0.6448 - acc: 0.7478 - val_loss: 0.5770 - val_acc: 0.7736
Epoch 2/4
105000/105000 [==============================] - 35s 333us/step - loss: 0.5450 - acc: 0.7908 - val_loss: 0.5485 - val_acc: 0.7875
Epoch 3/4
105000/105000 [==============================] - 35s 334us/step - loss: 0.5080 - acc: 0.8074 - val_loss: 0.5981 - val_acc: 0.7661
Epoch 4/4
105000/105000 [==============================] - 35s 334us/step - loss: 0.4768 - acc: 0.8216 - val_loss: 0.5615 - val_acc: 0.7883


In [128]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
ooe_preds = [labels[i] for i in preds]

In [130]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(max_words, 128, input_length=maxlen))
model1.add(Conv1D(64, 5, activation='relu'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(4, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
          optimizer='rmsprop',
          metrics=['acc'])
model1.fit(x_train, y_train_owtca, epochs=2, batch_size=64, validation_data=(x_val, y_val_owtca))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 36s 347us/step - loss: 0.5428 - acc: 0.8013 - val_loss: 0.4585 - val_acc: 0.8316
Epoch 2/2
105000/105000 [==============================] - 34s 328us/step - loss: 0.4465 - acc: 0.8363 - val_loss: 0.4492 - val_acc: 0.8366


In [131]:
labels = [-2, -1, 0, 1]
preds = model1.predict_classes(x_test)
owtca_preds = [labels[i] for i in preds]

#### 测试

- 位置：3个，location_traffic_convenience, location_distance_from_business_district, location_easy_to_find
- 服务：4个，service_wait_time, service_waiters_attitude, service_parking_convenience, service_serving_speed
- 价格：3个，price_level, price_cost_effective, price_discount
- 环境：4个，environment_decoration, environment_noise, environment_space, environment_cleaness
- 菜品：4个，dish_portion, dish_taste, dish_look, dish_recommendation
- 其他：2个，others_overall_experience, others_willing_to_consume_again

In [132]:
test_data['location_traffic_convenience'] = ltc_preds
test_data['location_distance_from_business_district'] = ldfbd_preds
test_data['location_easy_to_find'] = letf_preds

test_data['service_wait_time'] = swt_preds
test_data['service_waiters_attitude'] = swa_preds
test_data['service_parking_convenience'] = spc_preds
test_data['service_serving_speed'] = sss_preds

test_data['price_level'] = pl_preds
test_data['price_cost_effective'] = pce_preds
test_data['price_discount'] = pd_preds

test_data['environment_decoration'] = ed_preds
test_data['environment_noise'] = en_preds
test_data['environment_space'] = es_preds
test_data['environment_cleaness'] = ec_preds

test_data['dish_portion'] = dp_preds
test_data['dish_taste'] = dt_preds
test_data['dish_look'] = dl_preds
test_data['dish_recommendation'] = dr_preds

test_data['others_overall_experience'] = ooe_preds
test_data['others_willing_to_consume_again'] = owtca_preds

In [133]:
test_data

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",-2,-2,-2,-2,1,-2,-2,0,...,0,-2,-2,1,1,1,-2,-2,1,1
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-1,0,-2,1,1,1
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",1,1,1,-2,0,-2,-2,0,...,-2,-2,-2,-2,-2,1,-2,-2,1,-1
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",1,1,-2,-2,1,1,-2,-2,...,1,-2,-2,-2,-2,1,-2,1,1,-2
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",-2,-2,-2,0,1,-2,-2,-2,...,1,-1,-2,1,-2,1,-2,-2,1,-2
5,5,"""尽管韩国烤肉店在无锡已经有很多家了，但因为味道好吃再加上在无锡很有人气，依旧挡不住新店的开...",-1,-2,1,-2,1,-2,-2,-1,...,1,-2,-2,-2,0,1,-2,1,1,-2
6,6,"""店铺在乙烯生活二区的西北角，旁边是国旅的旗舰店，门口就是红绿灯，挺好找的。因为这是两磅一的...",-2,-2,1,-2,1,-2,-2,1,...,-2,-2,-2,-2,1,1,-2,-2,1,-2
7,7,"""朋友聚会来滴，这个地方真心不错哇，又能撸串，又能唱歌，还能看现场歌手演唱。最主要是可以上台...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,-2,-2,1,1
8,8,"""超喜欢这家的面食，经常来吃。这家风格及产品都很像京城御面，有时会怀疑是不是同一家的。面条比...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,1,0,-2,-2,1,-2
9,9,"""广西阳朔，前一天晚上吃的特色啤酒鱼，就看到了隔壁有家螺蛳粉，第二天过来吃。个人比较喜欢粉，...",-2,-2,1,-2,-2,1,1,0,...,-2,-2,1,1,-2,1,-2,-2,1,-2


In [134]:
test_data.to_csv('data/submission.csv', index=False)